In [ ]:
pip install openai

In [ ]:
 pip install langchain

In [ ]:
 pip install langchain-pinecone

In [ ]:
pip install langchain_openai

In [ ]:
 pip install langchain_community

In [ ]:
pip install pinecone

In [ ]:
pip install openai-chat

In [ ]:
pip install google-search-results

In [ ]:
pip uninstall pinecone-client

In [ ]:
pip install pinecone-client>=3.0.0

In [ ]:
!pip install --quiet langchain openai pinecone-client==2.2.1 python-dotenv

In [ ]:
pip install --upgrade pinecone-client

In [ ]:
# All imports

import os
import requests
#import pinecone

#from pinecone import Pinecone, ServerlessSpec                         # rel: Start
from langchain.embeddings.openai import OpenAIEmbeddings              # rel: Step 2
from langchain.document_loaders import PyPDFLoader                    # rel: Step 3
from langchain.text_splitter import RecursiveCharacterTextSplitter    # rel: Step 3
from langchain.vectorstores import Pinecone as LC_Pinecone            # rel: Step 4
from langchain_openai import OpenAI                                   # rel: Step 5
from langchain.chains import RetrievalQA

In [ ]:
!pip install langchain openai wikipedia faiss-cpu scikit-learn

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.tsv to train.tsv
Saving test.tsv to test.tsv


In [ ]:
# Set up API keys and environment variables

OPENAI_API_KEY = " your api key here "
PINECONE_KEY = "your api key here"
PINECONE_REGION = "your region here"

os.environ["PINECONE_KEY"] = PINECONE_KEY
os.environ["PINECONE_REGION"] = PINECONE_REGION

In [ ]:
# ========================
# STEP 4: Load & preprocess LIAR dataset
# ========================
import pandas as pd

def load_liar_dataset(file_path):
    df = pd.read_csv(file_path, sep='\t', header=None, names=[
        "id", "label", "statement", "subject", "speaker", "job_title",
        "state", "party", "barely_true_counts", "false_counts", "half_true_counts",
        "mostly_true_counts", "pants_on_fire_counts", "context"
    ])
    return df[["statement", "label"]]

def preprocess_labels(df):
    binary_map = {
        'false': 'fake',
        'pants-fire': 'fake',
        'barely-true': 'fake',
        'half-true': 'real',
        'mostly-true': 'real',
        'true': 'real'
    }
    df['label'] = df['label'].map(binary_map)
    return df[df['label'].isin(['fake', 'real'])]

def get_train_test_split(train_path, test_path):
    train_df = preprocess_labels(load_liar_dataset(train_path))
    test_df = preprocess_labels(load_liar_dataset(test_path))
    return train_df, test_df




In [ ]:
# ========================
# STEP 5: Build Wikipedia RAG Vector Store
# ========================
from langchain.document_loaders import WikipediaLoader
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

def build_wiki_vector_store(search_terms, api_key):
    documents = []
    for term in search_terms:
        loader = WikipediaLoader(query=term)
        docs = loader.load()
        documents.extend(docs)

    embedding = OpenAIEmbeddings(openai_api_key=api_key)
    vectorstore = FAISS.from_documents(documents, embedding)
    return vectorstore


In [ ]:
# ========================
# STEP 6: Setup LangChain QA
# ========================
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

def setup_qa(vectorstore, api_key):
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=api_key)
    return RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

def ask_model(qa, claim):
    prompt = f"Is the following claim likely fake or real? Just say 'fake' or 'real'. Claim: {claim}"
    response = qa.run(prompt)
    return "fake" if "fake" in response.lower() else "real"


In [ ]:
# ========================
# STEP 7: Evaluation Metrics
# ========================
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, pos_label="fake"),
        "recall": recall_score(y_true, y_pred, pos_label="fake"),
        "f1_score": f1_score(y_true, y_pred, pos_label="fake")
    }

In [ ]:
# STEP 8: Run RAG Pipeline on LIAR dataset
# ========================

# Only run this if you uploaded `train.tsv` and `test.tsv`
try:
    train_df, test_df = get_train_test_split("train.tsv", "test.tsv")
    search_terms = ["Fake news", "Misinformation", "COVID-19 misinformation", "Political disinformation", "Social media"]
    vectorstore = build_wiki_vector_store(search_terms, OPENAI_API_KEY)
    qa = setup_qa(vectorstore, OPENAI_API_KEY)

    sample_test = test_df.sample(30, random_state=42)
    y_true = sample_test['label'].tolist()
    y_pred = [ask_model(qa, claim) for claim in sample_test['statement']]

    metrics = evaluate(y_true, y_pred)

    print("\n📊 Evaluation Metrics:")
    for k, v in metrics.items():
        print(f"{k.capitalize()}: {v:.2f}")
except:
    print("⚠️ Skipping LIAR dataset pipeline — train.tsv/test.tsv not uploaded.")


/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
<ipython-input-20-bad5f12ac380>:15: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langch


📊 Evaluation Metrics:
Accuracy: 0.70
Precision: 0.73
Recall: 0.69
F1_score: 0.71


In [ ]:
!pip install newspaper3k

In [ ]:
!pip install lxml_html_clean

In [ ]:
# ========================
# STEP 9: URL-based Fake News Checker (on-demand)
# ========================
from newspaper import Article
from langchain.prompts import PromptTemplate

def extract_text_from_url(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

TEMPLATE = """
You are a fact-checking assistant. Assess the credibility of the following content:

\"\"\"{content}\"\"\"

Respond with one of the following labels: "credible", "potentially misleading", or "fake".
Then briefly explain why.
"""

prompt = PromptTemplate.from_template(TEMPLATE)

def analyze_text(text):
    input_prompt = prompt.format(content=text[:2000])  # Truncate to fit context window
    response = llm.predict(input_prompt)

    if "credible" in response.lower():
        return "credible", response
    elif "misleading" in response.lower():
        return "potentially misleading", response
    else:
        return "fake", response


In [ ]:
# ========================
# STEP 10: Run on URL
# ========================
url = input("📥 Paste a URL to analyze: ").strip()
try:
    article_text = extract_text_from_url(url)
    verdict, explanation = analyze_text(article_text)

    print(f"\n🧾 Verdict: {verdict.upper()}\n")
    print("💬 Explanation:\n")
    print(explanation)
except Exception as e:
    print(f"❌ Error analyzing URL: {e}")

📥 Paste a URL to analyze: https://www.snopes.com/fact-check/clinton-votes-found-in-warehouse/


<ipython-input-28-66885b567865>:28: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(input_prompt)



🧾 Verdict: FAKE

💬 Explanation:

Label: Fake

Explanation: The content contains false information about pre-marked ballots for Hillary Clinton and other Democratic candidates being found in a warehouse in Ohio. The article is based on fabricated information and altered images, making it completely unreliable and fake. Additionally, the Christian Times Newspaper has a history of promoting conspiracy theories and false stories, further diminishing its credibility.


In [ ]:
# ========================
# STEP 10: Run on URL
# ========================
url = input("📥 Paste a URL to analyze: ").strip()
try:
    article_text = extract_text_from_url(url)
    verdict, explanation = analyze_text(article_text)

    print(f"\n🧾 Verdict: {verdict.upper()}\n")
    print("💬 Explanation:\n")
    print(explanation)
except Exception as e:
    print(f"❌ Error analyzing URL: {e}")

📥 Paste a URL to analyze: https://www.nytimes.com/2015/08/16/us/politics/att-helped-nsa-spy-on-an-array-of-internet-traffic.html?mtrref=www.nytimes.com&gwh=39731CD5654384DA101033E4C386CB44&gwt=pay

🧾 Verdict: POTENTIALLY MISLEADING

💬 Explanation:

Potentially misleading

While the content provides specific details and references to support its claims, it is important to note that the information is based on leaked N.S.A. documents and anonymous sources. Without official confirmation or verification from the government or telecom companies involved, the claims made in the content could be potentially misleading.
